In [112]:
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
import osmnx
import openrouteservice
from folium.map import *
from openrouteservice import client
import numpy as np
from folium.plugins import HeatMapWithTime

In [ ]:
porto_area = osmnx.geocode_to_gdf('Porto, Portugal')

porto_area.plot()

center_porto = porto_area['geometry'].centroid


# openrouteservice api key
api_key = "5b3ce3597851110001cf6248516d2c7fa9b240a1989c451ecb3b2520"  
clnt = client.Client(key=api_key)

m = folium.Map(location=[41.16181,-8.62057], zoom_start=14)
m

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

TRAFFIC_DATA_PATH = [
    'traffic_flow_2018.csv',
    'traffic_flow_2019.csv'
]


def load_data(data_paths, date_col, value_cols, agg_operation = 'sum'):
    concat_data = []

    for file in data_paths:
        print("Reading file ", file)
        df = pd.read_csv('/content/drive/MyDrive/WDL/' + file, encoding='latin-1')
        
        # print(df.head())

        df[date_col] = pd.to_datetime(df[date_col])
        df_resampled = df.set_index(date_col).groupby('entity_id')[value_cols].resample('H').agg(agg_operation).reset_index()
        concat_data.append(df_resampled)

    return pd.concat(concat_data)

date_col = 'dateobservedfrom'
value_cols = ['intensity']
files_name = TRAFFIC_DATA_PATH
traffic_data = load_data(files_name, date_col, value_cols, agg_operation = 'sum')
traffic_data.rename(columns={'dateobservedfrom': 'dateobserved'}, inplace=True)

In [128]:
data_entities = pd.read_csv('/content/drive/MyDrive/WDL/data_entities.csv', sep=',', encoding='latin-1', error_bad_lines=False)
df_resampled = traffic_data.merge(data_entities, on='entity_id')
df_resampled.latitude = df_resampled.latitude.astype(float)
df_resampled.longitude = df_resampled.longitude.astype(float)

In [ ]:
df_resampled.pickup_datetime = pd.to_datetime(df_resampled.dateobserved, format='%Y-%m-%d %H:%M:%S')
df_resampled['month'] = df_resampled.dateobserved.apply(lambda x: x.month)
df_resampled['week'] = df_resampled.dateobserved.apply(lambda x: x.week)
df_resampled['day'] = df_resampled.dateobserved.apply(lambda x: x.day)
df_resampled['hour'] = df_resampled.dateobserved.apply(lambda x: x.hour)
df_resampled['year'] = df_resampled.dateobserved.apply(lambda x: x.year)

In [130]:
df_copy_2018 = df_resampled[df_resampled.year==2018]

df_month_list_2018 = []
for month in df_copy_2018.month.sort_values().unique():
    df_month_list_2018.append(df_copy_2018.loc[df_copy_2018.month == month, ['latitude', 'longitude', 'intensity']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist())

time_index_2018=['Jan-2018', 'Fev-2018', 'Mar-2018', 'April-2018', 'May-2018', 'June-2018', 'July-2018', 'Aug-2018', 'Sept-2018', 'Oct-2018', 'Nov-2018', 'Decemb-2018']

In [131]:
df_copy_2019 = df_resampled[df_resampled.year==2019]

df_month_list_2019 = []
for month in df_copy_2019.month.sort_values().unique():
    df_month_list_2019.append(df_copy_2019.loc[df_copy_2019.month == month, ['latitude', 'longitude', 'intensity']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist())

time_index_2019=['Jan-2019', 'Fev-2019', 'Mar-2019', 'April-2019', 'May-2019', 'June-2019', 'July-2019', 'Aug-2019', 'Sept-2019', 'Oct-2019', 'Nov-2019', 'Decemb-2019']

In [ ]:
#MESES 2018 E 2019

HeatMapWithTime(df_month_list_2018, index=time_index_2018, radius=8, gradient={0.2: 'blue', 
                            0.4: 'lime', 
                            0.7: 'orange',
                            0.9: '#FC3800',
                            1: 'red'}, min_opacity=0.8, max_opacity=1.0, use_local_extrema=True).add_to(m)


HeatMapWithTime(df_month_list_2019, index=time_index_2019, radius=8, gradient={0.2: 'blue', 
                            0.4: 'lime', 
                            0.7: 'orange',
                            0.9: '#FC3800',
                            1: 'red'}, min_opacity=0.8, max_opacity=1.0, use_local_extrema=True).add_to(m)
m

In [133]:
conditions = [
    (df_resampled['month'].isin([1, 2, 3])),
    (df_resampled['month'].isin([4, 5, 6])),
    (df_resampled['month'].isin([7, 8, 9])),
     (df_resampled['month'].isin([10, 11, 12])),
    ]

values = [1, 2, 3, 4]
df_resampled['trimester'] = np.select(conditions, values)

df_2018_tri = df_resampled[df_resampled.year==2018]
df_2019_tri = df_resampled[df_resampled.year==2019]

In [134]:
df_tri_list_2018 = []
for tri in df_2018_tri.trimester.sort_values().unique():
    df_tri_list_2018.append(df_2018_tri.loc[df_2018_tri.trimester == tri, ['latitude', 'longitude', 'intensity']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist())


time_index_tri_2018=['1Trim-2018', '2Trim-2018', '3Trim-2018', '4Trim-2018']

In [135]:
df_tri_list_2019 = []
for tri in df_2019_tri.trimester.sort_values().unique():
    df_tri_list_2019.append(df_2019_tri.loc[df_2019_tri.trimester == tri, ['latitude', 'longitude', 'intensity']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist())


time_index_tri_2019=['1Trim-2019', '2Trim-2019', '3Trim-2019', '4Trim-2019']

In [ ]:
#TRIMESTRES 2018 E 2019


HeatMapWithTime(df_tri_list_2018, index=time_index_tri_2018, radius=8, gradient={0.2: 'blue', 
                            0.4: 'lime', 
                            0.7: 'orange',
                            0.9: '#FC3800',
                            1: 'red'}, min_opacity=0.8, max_opacity=1.0, use_local_extrema=True).add_to(m)


HeatMapWithTime(df_tri_list_2019, index=time_index_tri_2019, radius=8, gradient={0.2: 'blue', 
                            0.4: 'lime', 
                            0.7: 'orange',
                            0.9: '#FC3800',
                            1: 'red'}, min_opacity=0.8, max_opacity=1.0, use_local_extrema=True).add_to(m)
m